# FastHTML and MonsterUI Time Converter

A FastHTML app that converts times to different timezones, and to Discord timestamps.

In [1]:
from datetime import datetime
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
from monsterui.all import *
from zoneinfo import ZoneInfo

## Bring Over Time Conversion Functions

This code is from [Functional Programming with datetime, and Omni-Timezone Discord Timestamps](https://audrey.feldroy.com/nbs/2025-01-28-Functional-Programming-with-datetime-and-Omni-Timezone-Discord-Timestamps).

In [2]:
#| export
tzs = L('America/Los_Angeles', 'America/Chicago', 'America/New_York', 'Europe/London', 'Europe/Istanbul', 'Australia/Brisbane',)
tzs

(#6) ['America/Los_Angeles','America/Chicago','America/New_York','Europe/London','Europe/Istanbul','Australia/Brisbane']

In [3]:
#| export
def time_in_tz(tz:str, dt:datetime|None=None) -> str: 
    if dt is None: dt = datetime.now()
    dt = dt.astimezone(ZoneInfo(tz))
    return f"{tz:20} {dt:%H:%M}"
tzs.map(time_in_tz).map(print)

America/Los_Angeles  07:49
America/Chicago      09:49
America/New_York     10:49
Europe/London        15:49
Europe/Istanbul      18:49
Australia/Brisbane   01:49


(#6) [None,None,None,None,None,None]

In [4]:
#| export
def print_discord_time(dt:datetime|None=None) -> None:
    if dt is None: dt = datetime.now()
    print(f"{"Discord":20} <t:{int(dt.timestamp())}:f>")
print_discord_time()

Discord              <t:1738597748:f>


In [5]:
#| export
def print_times(dt:datetime|None=None) -> None:
    if dt is None: dt = datetime.now()
    tzs = L('America/Los_Angeles', 'America/Chicago', 'America/New_York', 'Europe/London', 'Europe/Istanbul', 'Australia/Brisbane')
    tzs.map(time_in_tz).map(print)
    print_discord_time(dt)
print_times()

America/Los_Angeles  07:49
America/Chicago      09:49
America/New_York     10:49
Europe/London        15:49
Europe/Istanbul      18:49
Australia/Brisbane   01:49
Discord              <t:1738597748:f>


## Make a TimeConverter FastTag

In [8]:
app, rt = fast_app(hdrs=(Theme.slate.headers(),))

In [79]:
def TimeConverter():
    return Form(
        H3("Convert a Time", cls="text-base font-semibold text-gray-900 dark:text-slate-100"),
        Input(name="dt", type="datetime-local", value="2025-02-03T14:30"),
        Button("Submit", cls=ButtonT.primary), hx_post="/convert", hx_target="#grid", hx_swap="outerHTML")

show(TimeConverter())

In [70]:
dt = datetime.now()
dt

datetime.datetime(2025, 2, 3, 16, 8, 55, 368904)

In [71]:
def TimeTable(dt):
    tzs = L('America/Los_Angeles', 'America/Chicago', 'America/New_York', 'Europe/London', 'Europe/Istanbul', 'Australia/Brisbane',)
    return Table(
        Thead(Tr(Th(f"{dt:%a, %b %-d, %Y %H:%M} GMT is also", colspan=2))),
        *[Tr(
            Td(tz),
            Td(time_in_tz(tz, dt).split()[-1])
          ) for tz in tzs],
        Tr(Td("Discord"), Td(f"<t:{int(dt.timestamp())}:f>"))
    )

In [72]:
show(TimeTable(dt))

## Define Routes

In [76]:
@rt
def index():
    dt = datetime.now()
    return Grid(TimeTable(dt), TimeConverter(), id="grid")

In [74]:
@rt
def convert(dt:str):
    dt = datetime.fromisoformat(dt).replace(tzinfo=ZoneInfo('Europe/London'))
    return (TimeTable(dt), TimeConverter())

In [15]:
server = JupyUvi(app)

In [16]:
server.stop()

NameError: name 'server' is not defined